In [43]:
#Imports
import pandas as pd
import alpaca_trade_api as tradeapi
from pathlib import Path
import os
import json
from dotenv import load_dotenv

In [2]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# The Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [4]:
#Set Tickers for stocks and crypto for analysis only looking at top 10 existing for at least 4 years
stock_tickers = ['MSFT','AAPL','TSLA','AMZN','NVDA','GOOG','FB','ADBE','NFLX','PYPL']
crypto_tickers = ['BTC','ETH','BNB','ADA','XRP','DOGE','LTC','LINK','BCH','XLM']

In [46]:
# Setting time frame to 1D
timeframe = "1D"

# Set the start/ end datetime of 5 years from today 
start_date = pd.Timestamp('2018-02-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-23',tz='America/New_York').isoformat()

# Set limit of rows to the max to maximaize
limit_rows = 1000

# Get the 2 years data frame for stocks
stocks_df = alpaca.get_barset(
    stock_tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = limit_rows).df

# Visualazing the data 
stocks_df.head()

AAPL                                      \
                              open    high       low   close    volume   
time                                                                     
2018-02-01 00:00:00-05:00  167.165  168.62  166.7600  167.70  29765526   
2018-02-02 00:00:00-05:00  166.470  166.80  160.1000  160.41  67919941   
2018-02-05 00:00:00-05:00  159.100  163.88  156.0000  156.49  59470254   
2018-02-06 00:00:00-05:00  154.910  163.72  154.0000  163.06  54780528   
2018-02-07 00:00:00-05:00  163.085  163.40  159.0685  159.52  41035232   

                             ADBE                                     ...  \
                             open    high       low   close   volume  ...   
time                                                                  ...   
2018-02-01 00:00:00-05:00  199.12  201.75  198.0845  199.38  1532011  ...   
2018-02-02 00:00:00-05:00  197.33  199.40  195.4400  195.63  1410466  ...   
2018-02-05 00:00:00-05:00  194.06  198.46  188.0000  190.27  2332614  ...   
2018-02-06 00:00:00-05:00  186.59  194.82  182.1000  194.52  3130162  ...   
2018-02-07 00:00:00-05:00  193.87  196.24  192.0200  192.36  1714797  ...   

                            PYPL                                    TSLA  \
                            open    high    low  close    volume    open   
time                                                                       
2018-02-01 00:00:00-05:00  79.96  80.950  76.70  78.40  37352330  351.00   
2018-02-02 00:00:00-05:00  78.01  79.620  76.28  76.55  19063799  348.44   
2018-02-05 00:00:00-05:00  75.99  78.980  74.68  74.70  15685427  337.97   
2018-02-06 00:00:00-05:00  72.51  76.515  72.25  75.68  16194538  325.21   
2018-02-07 00:00:00-05:00  75.35  77.580  74.94  75.59   9497980  338.83   

                                                            
                             high     low   close   volume  
time                                                        
2018-02-01 00:00:00-05:00  359.66  348.63  349.25  3065835  
2018-02-02 00:00:00-05:00  351.95  340.51  343.75  2730271  
2018-02-05 00:00:00-05:00  344.47  333.00  333.15  3124163  
2018-02-06 00:00:00-05:00  336.22  323.50  333.97  3703402  
2018-02-07 00:00:00-05:00  346.00  335.66  344.77  4422730  

[5 rows x 50 columns]

In [49]:
#Create for loop to read in Change % data and clean column of string values and convert to float /100
crypto_daily_df = pd.DataFrame()
for crypto in crypto_tickers:
    df = pd.read_csv(Path(f"Crypto_Historical_Data/{crypto}.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
    #If dtype is O ---object then str.replace
    if df['Change %'].dtypes=='O':
        df['Change %'] = df['Change %'].str.replace('%','')
    else:
        df['Change %'] = df['Change %'].replace('%','')
    crypto_daily_df[f'{crypto}_%change'] = df['Change %'].astype('float').divide(100)

#drop na values or change to 0 depending on need
crypto_daily_df = crypto_daily_df.dropna()
crypto_daily_df.head()

,BTC_%change,ETH_%change,BNB_%change,ADA_%change,XRP_%change,DOGE_%change,LTC_%change,LINK_%change,BCH_%change,XLM_%change
Date,,,,,,,,,,
2021-04-25,-0.0225,0.0454,0.0137,-0.0111,-0.0155,-0.0700,0.0047,0.0236,-0.0185,-0.0211
2021-04-24,-0.0206,-0.0639,-0.0552,-0.0473,-0.1026,0.0856,-0.0671,-0.1119,-0.0800,-0.0560
2021-04-23,-0.0113,-0.0141,0.0410,0.0092,0.0098,-0.0463,-0.0458,-0.0029,-0.0177,0.0008
2021-04-22,-0.0388,0.0171,-0.0690,-0.0483,-0.1020,-0.1467,-0.0189,-0.0288,-0.0610,-0.0840
2021-04-21,-0.0471,0.0123,-0.0752,-0.0496,-0.0690,-0.0380,-0.0119,-0.0697,-0.0330,-0.0708


In [7]:
##Read in Crypto Data
# btc_df = pd.read_csv(Path("Crypto_Historical_Data/BTC.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# eth_df = pd.read_csv(Path("Crypto_Historical_Data/ETH.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# bnb_df = pd.read_csv(Path("Crypto_Historical_Data/BNB.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# ada_df = pd.read_csv(Path("Crypto_Historical_Data/ADA.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# xrp_df = pd.read_csv(Path("Crypto_Historical_Data/XRP.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# doge_df = pd.read_csv(Path("Crypto_Historical_Data/DOGE.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# ltc_df = pd.read_csv(Path("Crypto_Historical_Data/LTC.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# link_df = pd.read_csv(Path("Crypto_Historical_Data/LINK.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# bch_df = pd.read_csv(Path("Crypto_Historical_Data/BCH.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
# xlm_df = pd.read_csv(Path("Crypto_Historical_Data/XLM.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)